In [1]:
import os
import requests
import json
import numpy
import pandas as pd
import re

In [2]:
#access the COD Services API list of COD locations and themes
COD_External_URL = 'https://apps.itos.uga.edu/CODV2API/api/v1/themes/cod-ps/lookup/Get/2/aa/lka'
response = requests.get(COD_External_URL)
dataD = json.loads(json.dumps(response.json()))
dc = json.loads(json.dumps(dataD['data']))
df = pd.json_normalize(dc)
up = df.to_records()
f = pd.DataFrame(up)
f


,index,ADM2_PCODE,ADM2_EN,T_TL,T_00_04,T_05_09,T_10_14,T_15_19,T_20_24,T_25_29,T_30_34,T_35_39,T_40_44,T_45_49,T_50_54,T_55_59,T_60_64,T_65_69,T_70_74,T_75_79,T_80Plus,F_TL,F_00_04,F_05_09,F_10_14,F_15_19,F_20_24,F_25_29,F_30_34,F_35_39,F_40_44,F_45_49,F_50_54,F_55_59,F_60_64,F_65_69,F_70_74,F_75_79,F_80Plus,M_TL,M_00_04,M_05_09,M_10_14,M_15_19,M_20_24,M_25_29,M_30_34,M_35_39,M_40_44,M_45_49,M_50_54,M_55_59,M_60_64,M_65_69,M_70_74,M_75_79,M_80Plus
0,0,LK11,Colombo,2481877,159438,178880,168811,181175,199009,183181,160778,169460,181805,150993,146546,137869,133255,120003,94980,58767,56927,1266125,79313,89259,83043,86591,94889,88439,81136,87087,91001,76215,75048,72536,71003,65356,53686,34769,36754,1215752,80125,89621,85768,94584,104120,94742,79642,82373,90804,74778,71498,65333,62252,54647,41294,23998,20173
1,1,LK12,Gampaha,2449585,167903,185963,174901,183923,185734,179795,166207,167597,180287,147873,141416,134876,128242,111370,87363,54205,51930,1265110,83954,92473,86414,89308,89981,88195,85205,87953,92297,75418,73103,71330,69537,61932,50581,33209,34220,1184475,83949,93490,88487,94615,95753,91600,81002,79644,87990,72455,68313,63546,58705,49438,36782,20996,17710
2,2,LK13,Kalutara,1301124,95148,106238,96221,92482,87054,87363,84832,86613,92287,75451,72487,70290,69540,64320,54028,34448,32322,672564,47327,53052,47329,45045,42857,43731,44169,45146,47104,37960,37096,37048,37380,35227,30793,20469,20831,628560,47821,53186,48892,47437,44197,43632,40663,41467,45183,37491,35391,33242,32160,29093,23235,13979,11491
3,3,LK21,Kandy,1499319,114935,126283,118955,115613,104704,94735,88257,90121,101644,91376,88419,85562,81697,70486,59252,35019,32261,786011,57055,63136,58873,57702,54225,50371,48059,49154,53607,47293,45755,45097,43611,38429,32650,20722,20272,713308,57880,63147,60082,57911,50479,44364,40198,40967,48037,44083,42664,40465,38086,32057,26602,14297,11989
4,4,LK22,Matale,530766,43622,45581,41576,39718,34359,35768,33202,33956,36669,33443,32799,30807,27779,21957,18862,11160,9508,275509,21725,22802,21018,20114,17218,18625,17923,18066,19081,16857,16548,15993,14625,11989,10493,6649,5783,255257,21897,22779,20558,19604,17141,17143,15279,15890,17588,16586,16251,14814,13154,9968,8369,4511,3725
5,5,LK23,Nuwara Eliya,781498,67626,75095,67556,57291,49343,55320,49319,46283,51176,46725,46700,42435,40864,35603,25922,14216,10024,408196,33866,37608,33693,29534,25076,28787,26523,25053,26829,24431,24166,22384,22203,19663,14200,8125,6055,373302,33760,37487,33863,27757,24267,26533,22796,21230,24347,22294,22534,20051,18661,15940,11722,6091,3969
6,6,LK31,Galle,1147475,84243,93469,88760,88555,78850,74764,67009,69657,77723,65029,64557,62546,59321,57425,49105,33794,32668,598623,41863,46590,43467,43875,38875,38184,35603,37187,40619,33310,33318,33195,32165,31832,28152,20038,20350,548852,42380,46879,45293,44680,39975,36580,31406,32470,37104,31719,31239,29351,27156,25593,20953,13756,12318
7,7,LK32,Matara,875520,65721,71627,66604,67225,60753,56455,49221,52858,57025,49540,50590,50275,46536,42433,34652,26210,27795,456817,32821,35804,32948,33483,30297,29007,26259,28316,30016,25815,26088,26712,25064,22863,19300,14957,17067,418703,32900,35823,33656,33742,30456,27448,22962,24542,27009,23725,24502,23563,21472,19570,15352,11253,10728
8,8,LK33,Hambantota,675311,56905,58406,50243,53245,48541,49052,42208,42325,44653,39445,40047,40136,34041,24157,20054,15654,16199,344384,28491,29262,25231,26839,23869,24546,21887,21874,22652,19921,20033,20460,17512,12439,10867,8800,9701,330927,28414,29144,25012,26406,24672,24506,20321,20451,22001,19524,20014,19676,16529,11718,9187,6854,6498
9,9,LK41,Jaffna,627850,40912,50044,53897,56364,53565,44463,35808,34895,34489,31500,33213,34710,37474,31805,26309,15863,12539,333244,20656,25267,26869,27943,27267,23243,19277,19210,19303,17912,18430,19364,20544,17155,14335,8963,7506,294606,20256,24777,27028,28421,26298,21220,16531,15685,15186,13588,14783,15346,16930,14650,11974,6900,5033


In [3]:
def createDestination(path):
    try:
        os.makedirs(path)
    except:
        pass
destination = os.path.join(os.getcwd(),'hdxData')
i = "https://data.humdata.org/dataset/0bedcaf3-88cd-4591-b9d5-5d3220e26abf/resource/51a81e72-583c-407f-bce6-6f7b42431c93/download/lka_adm_20220816_shp.zip"
idx = i.rfind('/')
o = i[idx+1:len(i)]
fpath = o[0:len(o)-4]
path = os.path.join(destination,fpath)
createDestination(path)
fileName = os.path.basename(o)
outFile = os.path.join(path,o)

try:
    response = requests.get(i, stream=True)
    open(outFile, "wb").write(response.content)
except Exception as e:
    print('Error 404:',e)
    print('Destination:',path)    

In [5]:
import zipfile
aprx = arcpy.mp.ArcGISProject("current")
map = aprx.listMaps()[0]

with zipfile.ZipFile(outFile, 'r') as zip:
    print('Extracting shp....')
    zip.extractall()
    print('Done')
    shpDest = os.path.join(os.getcwd(),'lka_admbnda_adm2_slsd_20220816.shp')
    #zshp = zip.open(shpDest)

geoLayer = map.addDataFromPath(shpDest)


Extracting shp....
Done


In [18]:
layers = map.listLayers()
print (layers[0].name)

lka_admbnda_adm2_20220802_v01


In [19]:
map2 = aprx.listMaps()[0]
map2.listLayers()

[<arcpy._mp.Layer object at 0x000002B0951E0AC8>, <arcpy._mp.Layer object at 0x000002B0951E0F60>, <arcpy._mp.Layer object at 0x000002B0951E0B00>, <arcpy._mp.Layer object at 0x000002B0951E05C0>, <arcpy._mp.Layer object at 0x000002B0951E08D0>, <arcpy._mp.Layer object at 0x000002B021BB25F8>, <arcpy._mp.Layer object at 0x000002B021BB2710>]

In [28]:
#arrange output of the COD Services API to show the countries as rows

#df= pd.json_normalize(dataD)
#update_array = df.to_records()
recs = up #update_array
print(recs)
dts = {'names': ('id', 'ADM2_PCODE', 'ADM2_EN', 'T_TL', 'T_00_04', 'T_05_09', 'T_10_14', 'T_15_19', 'T_20_24', 'T_25_29', 'T_30_34', 'T_35_39', 'T_40_44', 'T_45_49', 'T_50_54', 'T_55_59', 'T_60_64', 'T_65_69', 'T_70_74', 'T_75_79', 'T_80Plus', 'F_TL', 'F_00_04', 'F_05_09', 'F_10_14', 'F_15_19', 'F_20_24', 'F_25_29', 'F_30_34', 'F_35_39', 'F_40_44', 'F_45_49', 'F_50_54', 'F_55_59', 'F_60_64', 'F_65_69', 'F_70_74', 'F_75_79', 'F_80Plus', 'M_TL', 'M_00_04', 'M_05_09', 'M_10_14', 'M_15_19', 'M_20_24', 'M_25_29', 'M_30_34', 'M_35_39', 'M_40_44', 'M_45_49', 'M_50_54', 'M_55_59', 'M_60_64', 'M_65_69', 'M_70_74', 'M_75_79', 'M_80Plus'), 'formats':(numpy.uint8, 'U13', '<S256', numpy.double, numpy.double, numpy.double, numpy.double, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8, numpy.uint8)}
array = numpy.rec.fromrecords(recs, dtype=dts)
df


[( 0, 'LK11', 'Colombo', 2481877, 159438, 178880, 168811, 181175, 199009, 183181, 160778, 169460, 181805, 150993, 146546, 137869, 133255, 120003, 94980, 58767, 56927, 1266125, 79313, 89259, 83043, 86591, 94889, 88439, 81136, 87087, 91001, 76215, 75048, 72536, 71003, 65356, 53686, 34769, 36754, 1215752, 80125, 89621, 85768, 94584, 104120, 94742, 79642, 82373, 90804, 74778, 71498, 65333, 62252, 54647, 41294, 23998, 20173)
 ( 1, 'LK12', 'Gampaha', 2449585, 167903, 185963, 174901, 183923, 185734, 179795, 166207, 167597, 180287, 147873, 141416, 134876, 128242, 111370, 87363, 54205, 51930, 1265110, 83954, 92473, 86414, 89308, 89981, 88195, 85205, 87953, 92297, 75418, 73103, 71330, 69537, 61932, 50581, 33209, 34220, 1184475, 83949, 93490, 88487, 94615,  95753, 91600, 81002, 79644, 87990, 72455, 68313, 63546, 58705, 49438, 36782, 20996, 17710)
 ( 2, 'LK13', 'Kalutara', 1301124,  95148, 106238,  96221,  92482,  87054,  87363,  84832,  86613,  92287,  75451,  72487,  70290,  69540,  64320, 54028

 (24, 'LK92', 'Kegalle',  900803,  66058,  72750,  66134,  64781,  57744,  56241,  54116,  54656,  60964,  54154,  55029,  55111,  53653,  46360, 38805, 23167, 21080,  472011, 32914, 36417, 32500, 32460, 28970, 28933, 29016, 29277, 31880, 27907, 28665, 29065, 29127, 25798, 21812, 13889, 13381,  428792, 33144, 36333, 33634, 32321,  28774, 27308, 25100, 25379, 29084, 26247, 26364, 26046, 24526, 20562, 16993,  9278,  7699)]


,ADM2_PCODE,ADM2_EN,T_TL,T_00_04,T_05_09,T_10_14,T_15_19,T_20_24,T_25_29,T_30_34,T_35_39,T_40_44,T_45_49,T_50_54,T_55_59,T_60_64,T_65_69,T_70_74,T_75_79,T_80Plus,F_TL,F_00_04,F_05_09,F_10_14,F_15_19,F_20_24,F_25_29,F_30_34,F_35_39,F_40_44,F_45_49,F_50_54,F_55_59,F_60_64,F_65_69,F_70_74,F_75_79,F_80Plus,M_TL,M_00_04,M_05_09,M_10_14,M_15_19,M_20_24,M_25_29,M_30_34,M_35_39,M_40_44,M_45_49,M_50_54,M_55_59,M_60_64,M_65_69,M_70_74,M_75_79,M_80Plus
0,LK11,Colombo,2481877,159438,178880,168811,181175,199009,183181,160778,169460,181805,150993,146546,137869,133255,120003,94980,58767,56927,1266125,79313,89259,83043,86591,94889,88439,81136,87087,91001,76215,75048,72536,71003,65356,53686,34769,36754,1215752,80125,89621,85768,94584,104120,94742,79642,82373,90804,74778,71498,65333,62252,54647,41294,23998,20173
1,LK12,Gampaha,2449585,167903,185963,174901,183923,185734,179795,166207,167597,180287,147873,141416,134876,128242,111370,87363,54205,51930,1265110,83954,92473,86414,89308,89981,88195,85205,87953,92297,75418,73103,71330,69537,61932,50581,33209,34220,1184475,83949,93490,88487,94615,95753,91600,81002,79644,87990,72455,68313,63546,58705,49438,36782,20996,17710
2,LK13,Kalutara,1301124,95148,106238,96221,92482,87054,87363,84832,86613,92287,75451,72487,70290,69540,64320,54028,34448,32322,672564,47327,53052,47329,45045,42857,43731,44169,45146,47104,37960,37096,37048,37380,35227,30793,20469,20831,628560,47821,53186,48892,47437,44197,43632,40663,41467,45183,37491,35391,33242,32160,29093,23235,13979,11491
3,LK21,Kandy,1499319,114935,126283,118955,115613,104704,94735,88257,90121,101644,91376,88419,85562,81697,70486,59252,35019,32261,786011,57055,63136,58873,57702,54225,50371,48059,49154,53607,47293,45755,45097,43611,38429,32650,20722,20272,713308,57880,63147,60082,57911,50479,44364,40198,40967,48037,44083,42664,40465,38086,32057,26602,14297,11989
4,LK22,Matale,530766,43622,45581,41576,39718,34359,35768,33202,33956,36669,33443,32799,30807,27779,21957,18862,11160,9508,275509,21725,22802,21018,20114,17218,18625,17923,18066,19081,16857,16548,15993,14625,11989,10493,6649,5783,255257,21897,22779,20558,19604,17141,17143,15279,15890,17588,16586,16251,14814,13154,9968,8369,4511,3725
5,LK23,Nuwara Eliya,781498,67626,75095,67556,57291,49343,55320,49319,46283,51176,46725,46700,42435,40864,35603,25922,14216,10024,408196,33866,37608,33693,29534,25076,28787,26523,25053,26829,24431,24166,22384,22203,19663,14200,8125,6055,373302,33760,37487,33863,27757,24267,26533,22796,21230,24347,22294,22534,20051,18661,15940,11722,6091,3969
6,LK31,Galle,1147475,84243,93469,88760,88555,78850,74764,67009,69657,77723,65029,64557,62546,59321,57425,49105,33794,32668,598623,41863,46590,43467,43875,38875,38184,35603,37187,40619,33310,33318,33195,32165,31832,28152,20038,20350,548852,42380,46879,45293,44680,39975,36580,31406,32470,37104,31719,31239,29351,27156,25593,20953,13756,12318
7,LK32,Matara,875520,65721,71627,66604,67225,60753,56455,49221,52858,57025,49540,50590,50275,46536,42433,34652,26210,27795,456817,32821,35804,32948,33483,30297,29007,26259,28316,30016,25815,26088,26712,25064,22863,19300,14957,17067,418703,32900,35823,33656,33742,30456,27448,22962,24542,27009,23725,24502,23563,21472,19570,15352,11253,10728
8,LK33,Hambantota,675311,56905,58406,50243,53245,48541,49052,42208,42325,44653,39445,40047,40136,34041,24157,20054,15654,16199,344384,28491,29262,25231,26839,23869,24546,21887,21874,22652,19921,20033,20460,17512,12439,10867,8800,9701,330927,28414,29144,25012,26406,24672,24506,20321,20451,22001,19524,20014,19676,16529,11718,9187,6854,6498
9,LK41,Jaffna,627850,40912,50044,53897,56364,53565,44463,35808,34895,34489,31500,33213,34710,37474,31805,26309,15863,12539,333244,20656,25267,26869,27943,27267,23243,19277,19210,19303,17912,18430,19364,20544,17155,14335,8963,7506,294606,20256,24777,27028,28421,26298,21220,16531,15685,15186,13588,14783,15346,16930,14650,11974,6900,5033


In [29]:
output_table = "CODCountries23"
arcpy.da.NumPyArrayToTable(array, os.path.join(arcpy.env.workspace, output_table))


In [30]:
#Load the features with geometry and corresponding ISO 3 country values
inFeatures = geoLayer
inField = "ADM2_PCODE"
joinTable = os.path.join(arcpy.env.workspace, output_table)
joinField = "ADM2_PCODE" 
outFeature = "MyProject18.gdb/PopStats"
jt = arcpy.AddJoin_management(inFeatures, inField, joinTable, joinField)
print (jt)
arcpy.CopyFeatures_management(jt, outFeature)
aprx = arcpy.mp.ArcGISProject("current")
map = aprx.listMaps()[0]
layers = map.listLayers()
newLayer = layers[0]
sym = newLayer.symbology
sym.updateRenderer("GraduatedColorsRenderer")
sym.valueField = "T_TL"
r = sym.renderer
fields = ['ADM2_PCODE']
r.breakCount = 5
r.colorRamp = aprx.listColorRamps('Cyan to Purple') [0]
r.fields = "['T_TL']"#['adm1_polygons.CODCountries10.iso3']
print(r.fields)
r.classificationField = "CODCountries23_T_TL"
sym.renderer = r
#sym.renderer.fields = "['T_TL']" #['adm1_polygons.CODCountries10.iso3']
print (r.fields)
#sym.addAllValues()
#for grp in sym.renderer.groups:
#    for item in grp.items:
#        value = item.values[0][0]
#        if value == ' ':
#            item.symbol.color = {'RGB': [255, 12, 12, 100]}
newLayer.symbology = sym

lka_admbnda_adm2_20220802_v01
['T_TL']
['T_TL']
